In [ ]:
pip install cimcb

In [31]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import cimcb as cb

print('All packages successfully loaded')

All packages successfully loaded


In [32]:
seed_split = 100
seed_init = 4
# seed_split = None
# seed_init = None

In [33]:
from google.colab import files

uploaded = files.upload()

Saving Data and Peak sheet_Test.xlsx to Data and Peak sheet_Test.xlsx


In [34]:
# The path to the input file (Excel spreadsheet)
filename = 'Data and Peak sheet_Test.xlsx'
#filename = 'MTBLS290db.xlsx'

# Load Peak and Data tables into two variables
dataTable, peakTable = cb.utils.load_dataXL(filename, DataSheet='Data', PeakSheet='Peaks')

Loadings PeakFile: Peaks
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 418 TOTAL PEAKS: 8765
Done!


In [35]:
# Clean PeakTable and Extract PeakList
RSD = peakTable['QC_RSD']   
percMiss = peakTable['Perc_missing']  
peakTableClean = peakTable[(RSD < 20) & (percMiss < 10)]   
peakList = peakTableClean['Name']  

print("Number of peaks remaining: {}".format(len(peakTableClean)))

# Select Subset of Data (Class "MD" or "CG" only)
dataTable2 = dataTable[(dataTable.Class == "MD") | (dataTable.Class == "CG")]

# Extract X Data
X = dataTable2[peakList]  

# Create a Binary Y Vector 
outcomes = dataTable2['Class']                                  
Y = [1 if outcome == 'MD' else 0 for outcome in outcomes]         
Y = np.array(Y)   

# Optional: Save Class Labels for Figure Legends
Class = dataTable2.Class

Number of peaks remaining: 2259


In [36]:
# Split Data into Train (2/3rd) and Test (1/3rd)
XTrain, XTest, YTrain, YTest, ClassTrain, ClassTest = train_test_split(X,
                                                                       Y,
                                                                       Class,
                                                                       test_size=1/3,
                                                                       stratify=Y,
                                                                       random_state=seed_split)

In [37]:
# Extract X Train Data                          
XTrainLog = np.log(XTrain)                                          
XTrainScale= cb.utils.scale(XTrainLog, method='auto')              
XTrainKnn = cb.utils.knnimpute(XTrainScale, k=3)   

# Extract X Test Data
XTestLog = np.log(XTest)
XTestScale = cb.utils.scale(XTestLog, method='auto')
XTestKnn = cb.utils.knnimpute(XTestScale, k=3)

In [38]:
# Parameter Dictionary
lr = [0.01,0.02,0.03,0.04,0.05]
neurons = [2, 3, 4, 5, 6]

param_dict = dict(learning_rate=lr,
                  n_neurons=neurons,
                  epochs=400,
                  momentum=0.5,
                  decay=0,
                  loss='binary_crossentropy')

# Initialise
cv = cb.cross_val.kfold(model=cb.model.NN_SigmoidSigmoid,
                                X=XTrainKnn,
                                Y=YTrain,
                                param_dict=param_dict,
                                folds=5,
                                n_mc=10)

# Run 
cv.run()

Number of cores set to: 2
Running ...


100%|██████████| 250/250 [55:23<00:00, 13.29s/it]


Time taken: 55.94 minutes with 2 cores
Done!


In [39]:
cv.plot(metric='auc', ci=95)
cv.plot(metric='r2q2', ci=95)

/usr/local/lib/python3.7/dist-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/usr/local/lib/python3.7/dist-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/usr/local/lib/python3.7/dist-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()


/usr/local/lib/python3.7/dist-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/usr/local/lib/python3.7/dist-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/usr/local/lib/python3.7/dist-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()


In [40]:
cv.plot_projections()

In [41]:
# Build Model
model = cb.model.NN_LogitLogit(learning_rate=0.03,
                                   n_neurons=3,
                                   epochs=400,
                                   momentum=0.5,
                                   decay=0,
                                   loss='binary_crossentropy')
YPredTrain = model.train(XTrainKnn, YTrain)
YPredTest = model.test(XTestKnn)

# Put YTrain and YPredTrain in a List
EvalTrain = [YTrain, YPredTrain]

# Put YTest and YPrestTest in a List
EvalTest = [YTest, YPredTest]


# Evaluate Model (include Test Dataset)
model.evaluate(testset=EvalTest)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [42]:
model.permutation_test(nperm=100)

INFO:tensorflow:Assets written to: ram://1151c2e6-8fd2-453b-8337-c93e9f2f87d0/assets


Permutation Resample: 100%|██████████| 100/100 [45:04<00:00, 27.04s/it]


In [43]:
from keras.utils.generic_utils import default
# Extract X Data and Train Model
XBoot = dataTable2[peakList]
XBootLog = np.log(XBoot)
XBootScale = cb.utils.scale(XBootLog, method='auto')
XBootKnn = cb.utils.knnimpute(XBootScale, k=3)
YPredBoot = model.train(XBootKnn, Y)

# Build Boostrap Models
bootmodel = cb.bootstrap.BCA(model, XBootKnn, Y, bootlist=YPredBoot, bootnum=100)
bootmodel.run()

/usr/local/lib/python3.7/dist-packages/cimcb/utils/knnimpute.py:92: RuntimeWarning: divide by zero encountered in true_divide
  weights = 1 / dist[1 : k + L + 1, nan_cols[i]]
/usr/local/lib/python3.7/dist-packages/cimcb/utils/wmean.py:36: RuntimeWarning: Mean of empty slice
  m = np.nanmean(x[infs])


TypeError: ignored